In [265]:
#Importing relevant libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist


In [266]:
#Loading the mnist Dataset
(train_data, train_targets), (test_data, test_targets) = mnist.load_data()
print(train_data.shape )
print(train_targets.shape) 
print(test_data.shape )
print(test_targets.shape) 

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [267]:
#Number of samples needed during validation
num_validation_samples = 0.1*train_data.shape[0]
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
print(num_validation_samples)

tf.Tensor(6000, shape=(), dtype=int64)


In [268]:
#Number of samples
num_test_samples = train_data.shape[0]


In [269]:
#Scaling the data to be between 0 and 1 as each pixel has a value between 0 and 255
scaled_train_data=train_data/255
scaled_test_data=test_data/255
input_shape = scaled_train_data[0].shape

In [270]:
#Converting the numpy arrays into one tensorflow dataswt with label
train_dataset = tf.data.Dataset.from_tensor_slices((scaled_train_data, train_targets))
test_dataset = tf.data.Dataset.from_tensor_slices((scaled_test_data, test_targets))

In [271]:
#Shuffling the data
BUFFER_SIZE = 10000#Neccesary as a large dataset(hyperparameter that can be fine tuned)

shuffled_train_and_valida_data = train_dataset.shuffle(BUFFER_SIZE)

In [272]:

#Creating validation and training data
validation_data =shuffled_train_and_valida_data.take(num_validation_samples)
train_data = shuffled_train_and_valida_data.skip(num_validation_samples)

BATCH_SIZE = 100 #Hyperparameter that can be changed to fine tune the algorithimn
#1 would be the stochastic gradient descent


train_data = train_data.batch(BATCH_SIZE)#Only used on training data since validation data is backpropogated
validation_data = validation_data.batch(num_validation_samples)#Has to use validation data at once
test_data = test_dataset.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [273]:
#Outlining the model
input_size= 784
output_size =10
hidden_layer_1 = 392
hidden_layer_2 = 150
hidden_layer_3 = 50
model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=input_shape),
                            tf.keras.layers.Dense(hidden_layer_1, activation='relu'),#Hyperparameter to play around with
                            tf.keras.layers.Dense(hidden_layer_2, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_3, activation='tanh'),
      
                            tf.keras.layers.Dense(output_size, activation='softmax'),
])


In [274]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [275]:
NUM_EPOCHS = 7

#model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs,validation_targets),verbose = 2)
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=validation_data,validation_steps=1)

Train for 540 steps, validate for 1 steps
Epoch 1/7
540/540 [==============================] - 3s 5ms/step - loss: 0.2423 - accuracy: 0.9294 - val_loss: 0.1069 - val_accuracy: 0.9685
Epoch 2/7
540/540 [==============================] - 2s 4ms/step - loss: 0.0908 - accuracy: 0.9730 - val_loss: 0.0787 - val_accuracy: 0.9745
Epoch 3/7
540/540 [==============================] - 2s 4ms/step - loss: 0.0583 - accuracy: 0.9823 - val_loss: 0.0651 - val_accuracy: 0.9813
Epoch 4/7
540/540 [==============================] - 2s 4ms/step - loss: 0.0439 - accuracy: 0.9861 - val_loss: 0.0472 - val_accuracy: 0.9867
Epoch 5/7
540/540 [==============================] - 2s 4ms/step - loss: 0.0315 - accuracy: 0.9904 - val_loss: 0.0384 - val_accuracy: 0.9863
Epoch 6/7
540/540 [==============================] - 2s 4ms/step - loss: 0.0268 - accuracy: 0.9917 - val_loss: 0.0302 - val_accuracy: 0.9903
Epoch 7/7
540/540 [==============================] - 2s 4ms/step - loss: 0.0228 - accuracy: 0.9928 - val_loss: 0

In [276]:
#Test model
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 97ms/step - loss: 0.0809 - accuracy: 0.9771


In [277]:
print("Test loss: {0:.2f}. Test accuracy: {1:.2f}%".format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.71%


In [278]:
model.save('First_attempt_at_Mnist.h5')